In [1]:
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound, TranscriptsDisabled, VideoUnavailable
from pytubefix import YouTube
from pytubefix.cli import on_progress
import whisper
import torch
import os
import re

# 디바이스 설정
device =  torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:


# Whisper 모델 로드
model = whisper.load_model("large", device=device)

def get_title_hash(url):
    try:
        # YouTube 객체 생성
        yt = YouTube(url)

        # 1. 영상 제목 추출
        title = yt.title

        # 2. 해시태그 추출
        description = yt.description
        hashtags = re.findall(r"#\w+", description)
        hashtags = " ".join(hashtags)
        
        return {
            'title': title,
            'hashtags': hashtags
        }
    except Exception as e:
        return f"Error: {str(e)}"

def get_script(url, language='ko'):
    try:
        # 유튜브 영상 ID 추출
        video_id = url.split("=")[1]

        # 1. 자막 추출 시도
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
            return transcript
        except (NoTranscriptFound, TranscriptsDisabled):
            print("자막이 없거나 비활성화되어 있습니다. 음성 추출을 시도합니다.")

        # 2. 자막이 없는 경우, 음성 추출 및 Whisper 사용
        # 유튜브 영상에서 오디오 스트림 다운로드
        yt = YouTube(url, on_progress_callback=on_progress)
        audio_stream = yt.streams.filter(only_audio=True).first()
        
        # 오디오 데이터를 파일로 저장
        temp_audio_file = "temp_audio.wav"  # 임시로 사용할 파일 이름
        audio_stream.download(output_path='.', filename=temp_audio_file)

        # 오디오 파일을 Whisper 모델에 입력하여 텍스트 변환 수행
        result = model.transcribe(temp_audio_file, language=language)

        # Whisper 결과에서 각 세그먼트를 사용하여 시간 정보와 텍스트 추출
        checked_text = set()
        transcript_with_timestamps = []
        for segment in result['segments']:
            start = segment['start']  # 시작 시간 (초 단위)
            end = segment['end']      # 종료 시간 (초 단위)
            text = segment['text']    # 텍스트
            if text not in checked_text:
                checked_text.add(text)

                # 시작 시간과 지속 시간을 계산하여 딕셔너리 형식으로 저장
                transcript_with_timestamps.append({
                    'text': text.strip(),
                    'start': start,
                    'duration': end - start
                })
            else:
                pass

        # 3. 임시 파일 삭제
        if os.path.exists(temp_audio_file):
            os.remove(temp_audio_file)

        # 변환된 텍스트 반환
        return transcript_with_timestamps

    except VideoUnavailable:
        return "Error: 이 영상을 찾을 수 없습니다. 링크가 올바른지 확인하세요."
    except Exception as e:
        return f"Error: 알 수 없는 오류가 발생했습니다. {str(e)}"
    
def main(url):
    # 1. 영상 제목 추출
    title_hash = get_title_hash(url)
    script = get_script(url)
    return title_hash, script


In [22]:
title, script = main("https://youtu.be/HsxWL3kDYh8?si=aGGS9vmvp6dNX8P2")
print(title,"\n")
for _ in script:
    print(_['text'])

자막이 없거나 비활성화되어 있습니다. 음성 추출을 시도합니다.
{'title': '아메리칸 스타일로 부탁해', 'hashtags': '#소개팅앱 #소개팅 #데이트 #만남'} 

안녕하세요 여러분
오늘
앱블리케이션 소개령이 있는 날이에요
저는 정말 기대되고 있어요
예쁜 여자가
내 동네까지 픽업 왔어요
한국여자 정말 너무 스윗해요
후우
가볼게요
안녕하세요
오 마이 갓
저는 업다운 리이고
텍사스에서 왔어요
솔직히
앱블리케이션 만나면 하는 거 처음이에요
긴장돼요
잘 부탁해요
귀엽네
옷 벗고 뒤에 타
배꼽부터 한번 맞춰보자
오 배꼽?
왜? 뒤에 타라니까
죄송해요
잘 이해 못했어요
뒤에서 업다운 운동 좀 하자고
바운스 바운스 투게더 오케이?
No it's not okay
No no 그거 아니에요
그거 안돼요
미국에서 왔다며
아메리칸 스타일로 시원하게 한번 하자는데
그게 그렇게 어려워?
그냥 스포츠 한 게임 뛴다고 생각해
Oh calm down please stop
왜?
미국에서도 이렇게 냅다꼽진 않아요
Maybe?
혹시 내 돈 줘야 돼요?
아 혹시 내 받는 거야?
혹시 내 받는 거야?
뭐?
Okay stop
I got it
뒤로 가면 되죠
Fuck off bitch
야 너 폰
Okay okay
You know
나는
좀 서로 좀
알아가고 싶어요
그래
알아보자는 거잖아
나 오늘 파스타 맛집도 알아놨고
and then
아 진짜 잘하네
뭐 먹고 뭐 할 건데?
커피 마시고 싶고
커피 마시고 뭐 할 건데?
분위기 좋은 bar 가서
술 마시고 뭐 할 건데?
아 씨X야 어차피 할 거잖아
시간도 없는데 진짜 짜증나게
야 너 안 할 거야?
안 할 거야? 안 할 거냐고
할 거잖아
I mean 해야 할 것 같긴 한데요
요리 완성되어 있는 그런 느낌이란 말이에요
그럼 그냥 먹어
I don't know about it
야
야 갖고 꺼져
야 가 꺼져 꺼져
아 씨
아 씨X야
Okay let's go
Right now right here bounce bounce
Let's go bounce bounc

In [2]:
from faster_whisper import WhisperModel
model = WhisperModel("large-v3",device="cuda",compute_type="float16")

/home/jinu/miniconda3/envs/youtubee/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_script(url, language='ko'):
    # 유튜브 영상 ID 추출
    video_id = url.split("=")[1]

    # 1. 자막 추출 시도
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
        return transcript
    except (NoTranscriptFound, TranscriptsDisabled):
        print("자막이 없거나 비활성화되어 있습니다. 음성 추출을 시도합니다.")

    # 2. 자막이 없는 경우, 음성 추출 및 Whisper 사용
    # 유튜브 영상에서 오디오 스트림 다운로드
    yt = YouTube(url, on_progress_callback=on_progress)
    audio_stream = yt.streams.filter(only_audio=True).first()
    
    # 오디오 데이터를 파일로 저장
    temp_audio_file = "temp_audio.wav"  # 임시로 사용할 파일 이름
    audio_stream.download(output_path='.', filename=temp_audio_file)

get_script("https://youtu.be/HsxWL3kDYh8?si=aGGS9vmvp6dNX8P2")

자막이 없거나 비활성화되어 있습니다. 음성 추출을 시도합니다.


In [9]:
segments, info = model.transcribe("temp_audio.wav", beam_size=5)

In [10]:
for segment in segments:
    print(segment.start,    segment.end,    segment.text)

0.0 7.68  Hello my name is Luis Serrano and in this video you're going to learn about the math behind
7.68 13.26  attention mechanisms in large language models. Attention mechanisms are really important in
13.26 17.44  large language models. As a matter of fact they're one of the key steps that make transformers work
17.44 23.52  really well. Now in a previous video I showed you how attention works in very high level with words
23.52 28.12  flying towards each other and gravitating towards other words in order for the model to understand
28.12 33.5  context. In this video we're going to do an example in much more detail with all the math involved.
34.3 38.5  Now as I mentioned before the concept of a transformer and the attention mechanism were
38.5 43.120000000000005  introduced in this groundbreaking paper called attention is all you need. Now this is a series
43.120000000000005 47.52  of three videos. In the first video I showed you what attention mechanisms are in high level.
47.84

KeyboardInterrupt: 